In [1]:
import os
import requests
from PIL import Image
from glob import glob
import json



# === 사용자 설정 ===
prediction_key = "5sloqMYWiZSpfCd7HZgQ9ZpfuQAXnRWwjSw648WjXGr5Fy5f7imcJQQJ99BFACYeBjFXJ3w3AAAIACOGBDCR"
prediction_url = "https://7aiteam05ai012-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/f360eeb2-f8df-441e-8bed-f27d3ef1797a/detect/iterations/Iteration2/image"
input_folder = "C:/Users/USER/Desktop/code_project1/project/new_heavy"
output_folder = "C:/Users/USER/Desktop/code_project1/project/new_heavy_crop"
fail_folder = os.path.join(output_folder, "fail")
confidence_threshold = 0.5

# === 폴더 생성 ===
os.makedirs(output_folder, exist_ok=True)
os.makedirs(fail_folder, exist_ok=True)

# === 이미지 경로 불러오기 ===
image_paths = glob(os.path.join(input_folder, "*.jpg"))

# === 예측 + 크롭 ===
for image_path in image_paths:
    try:
        # 이미지 열어서 크기 체크
        img = Image.open(image_path)
        width, height = img.size

        if width < 100 or height < 100:
            print(f"⚠️ 너무 작은 이미지 건너뜀: {os.path.basename(image_path)}")
            img.save(os.path.join(fail_folder, os.path.basename(image_path)))
            continue

        # 예측 요청
        with open(image_path, "rb") as img_file:
            headers = {
                "Prediction-Key": prediction_key,
                "Content-Type": "application/octet-stream"
            }
            response = requests.post(prediction_url, headers=headers, data=img_file.read())

        try:
            response_json = response.json()
        except Exception:
            print(f"❌ JSON 파싱 실패: {os.path.basename(image_path)}")
            with open(os.path.join(fail_folder, os.path.basename(image_path)), "wb") as f:
                f.write(open(image_path, "rb").read())
            continue

        # 예측 결과 확인
        if "predictions" not in response_json:
            print(f"❌ 예측 실패: {os.path.basename(image_path)}")
            print("📋 응답 내용:", response_json)

            # 실패 이미지 저장
            img.save(os.path.join(fail_folder, os.path.basename(image_path)))

            # 응답 JSON도 같이 저장
            json_path = os.path.join(fail_folder, os.path.splitext(os.path.basename(image_path))[0] + ".json")
            with open(json_path, "w", encoding="utf-8") as jf:
                json.dump(response_json, jf, ensure_ascii=False, indent=2)

            continue

        predictions = response_json["predictions"]
        cropped_count = 0

        for i, pred in enumerate(predictions):
            if pred["probability"] < confidence_threshold:
                continue

            box = pred["boundingBox"]
            left = int(box["left"] * width)
            top = int(box["top"] * height)
            right = int((box["left"] + box["width"]) * width)
            bottom = int((box["top"] + box["height"]) * height)

            cropped = img.crop((left, top, right, bottom))
            filename = os.path.splitext(os.path.basename(image_path))[0]
            cropped.save(os.path.join(output_folder, f"{filename}_crop{i}.jpg"))
            cropped_count += 1

        print(f"✅ {os.path.basename(image_path)} → {cropped_count}개 크롭 완료")

    except Exception as e:
        print(f"⚠️ {os.path.basename(image_path)} 처리 중 오류 발생: {e}")


✅ 01.37183299.1.jpg → 2개 크롭 완료
✅ 1033143_395683_245.jpg → 1개 크롭 완료
✅ 119948608.8.jpg → 1개 크롭 완료
✅ 15077_15667_429.jpg → 1개 크롭 완료
✅ 20160705515756.jpg → 1개 크롭 완료
✅ 2018070111393032714_1530412769.jpg → 1개 크롭 완료
✅ 20190513172132.jpg → 1개 크롭 완료
✅ 202308101509092496_l.jpg → 2개 크롭 완료
✅ 20240221580234.jpg → 1개 크롭 완료
✅ 2180278_616468_1909.jpg → 1개 크롭 완료
✅ 403748_184392_5919.jpg → 1개 크롭 완료
✅ 5380_6064_1417.jpg → 1개 크롭 완료
✅ 63482_34000_5327.jpg → 1개 크롭 완료
⚠️ 63482_34002_317.jpg 처리 중 오류 발생: cannot write mode RGBA as JPEG
✅ 701067_374139_2424.jpg → 1개 크롭 완료
✅ 8102_22849_1021.jpg → 1개 크롭 완료
✅ 927895_330533_4341.jpg → 2개 크롭 완료
✅ 955768_646815_541.jpg → 1개 크롭 완료
✅ db222b86-092f-41f2-bacf-53c1f2979e0f.jpg → 2개 크롭 완료
✅ download.jpg → 2개 크롭 완료
✅ IE001065926_STD.jpg → 1개 크롭 완료
✅ images (2).jpg → 1개 크롭 완료
✅ images (3).jpg → 2개 크롭 완료
✅ images (4).jpg → 1개 크롭 완료
✅ images.jpg → 2개 크롭 완료
✅ maxresdefault.jpg → 1개 크롭 완료
✅ PYH2011090701900000401_P4.jpg → 1개 크롭 완료
✅ PYH2011090701910000401_P4.jpg → 1개 크롭 완료
✅ sosp